In [1]:
from sklearn.datasets import load_files
from sklearn.metrics import matthews_corrcoef, make_scorer
from sklearn.model_selection import cross_validate
from keras.utils import np_utils
from keras.preprocessing import image                  
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from keras.layers import Activation, BatchNormalization, Flatten, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras import optimizers
from tqdm import tqdm
import numpy as np

Using TensorFlow backend.


In [2]:
def load_dataset(path):
    data = load_files(path)
    spectogram_files = np.array(data['filenames'])
    spectogram_targets = np_utils.to_categorical(np.array(data['target']), 2)
    return spectogram_files, spectogram_targets

In [3]:
train_files, train_targets = load_dataset('Data/train_spectograms/')

In [4]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path)
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [5]:
train_tensors = paths_to_tensor(train_files).astype('float32')/255

100%|██████████| 2625/2625 [00:03<00:00, 797.93it/s]


In [6]:
learning_rate = 0.005
train_epochs=15

In [7]:
def cnn_model():
    # Define the network
    model = Sequential()

    model.add(Conv2D(16, 3, padding='same', use_bias=False, input_shape=(224, 224, 3)))
    model.add(BatchNormalization(axis=3))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
    model.add(Dropout(0.2))

    model.add(Conv2D(32, 3, padding='same', use_bias=False))
    model.add(BatchNormalization(axis=3))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, 3, padding='same', use_bias=False))
    model.add(BatchNormalization(axis=3))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, 3, padding='same', use_bias=False))
    model.add(BatchNormalization(axis=3))
    model.add(Activation("relu"))
    model.add(Dropout(0.2))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [8]:
model = cnn_model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 16)      432       
_________________________________________________________________
batch_normalization_1 (Batch (None, 224, 224, 16)      64        
_________________________________________________________________
activation_1 (Activation)    (None, 224, 224, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 56, 56, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)    

In [9]:
neural_network = KerasClassifier(build_fn=cnn_model, epochs=train_epochs, batch_size=32, verbose=1)

In [10]:
scoring = {'accuracy': 'accuracy', 'recall': 'recall', 
           'precision': 'precision', 'f1': 'f1', 'mcc': make_scorer(matthews_corrcoef)}

In [ ]:
cv_results = cross_validate(neural_network, train_tensors, train_targets, scoring=scoring, return_train_score=True)

/anaconda3/envs/py3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Instructions for updating:
Use tf.cast instead.
Epoch 1/15
1750/1750 [==============================] - 49s 28ms/step - loss: 0.3342 - acc: 0.8469
Epoch 2/15
1750/1750 [==============================] - 48s 27ms/step - loss: 0.2830 - acc: 0.8766
Epoch 3/15
1750/1750 [==============================] - 48s 27ms/step - loss: 0.2603 - acc: 0.8857
Epoch 4/15
1750/1750 [==============================] - 48s 27ms/step - loss: 0.2653 - acc: 0.8794
Epoch 5/15
1750/1750 [==============================] - 48s 27ms/step - loss: 0.2548 - acc: 0.8903
Epoch 6/15
1750/1750 [==============================] - 48s 27ms/step - loss: 0.2440 - acc: 0.8977
Epoch 7/15
1750/1750 [==============================] - 48s 28ms/step - loss: 0.2342 - acc: 0.9011
Epoch 8/15
1750/1750 [==============================] - 49s 28ms/step - loss: 0.2307 - acc: 0.8960
Epoch 9/15
1750/1750 [==============================] - 48s 27ms/step - loss: 0.2311 - acc: 0.8994
Epoch 10/15
1750/1750 [==============================] - 48s 